In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here'sta several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from numpy import math

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, adjusted_rand_score
import matplotlib.pyplot as plt

data = pd.read_csv("/kaggle/input/pricex/pricex.csv", sep=';')
data.head()

,Airplane,Date,DepCode,Way,DesCode,DepTime,ArrTime,Day,Duration,Price,PriceEx,DateOfBought,Stop,Days
0,THY,1.06.2022,ESB,ESB-IST-AYT,AYT,03:05,07:30,A,265,789,98,16.02.2022,2.0,105
1,THY,1.06.2022,ESB,ESB-IST-GZT,GZT,03:05,08:15,A,310,909,98,16.02.2022,2.0,105
2,THY,1.06.2022,ESB,ESB-DLM,DLM,07:50,09:00,A,70,606,99,16.02.2022,1.0,105
3,THY,1.06.2022,ESB,ESB-IST,IST,03:05,04:15,A,70,388,99,16.02.2022,1.0,105
4,THY,1.06.2022,ESB,ESB-IST,IST,07:50,08:55,A,65,539,99,16.02.2022,1.0,105


In [2]:
data.DateOfBought = data.DateOfBought.str.split(".")
data["BoughtDay"] = data.DateOfBought.str[0]
data["BoughtMonth"] = data.DateOfBought.str[1]
data["BoughtYear"] = data.DateOfBought.str[2]

data.Date = data.Date.str.split(".")
data["FlightDay"] = data.Date.str[0]
data["FlightMonth"] = data.Date.str[1]
data["FlightYear"] = data.Date.str[2]

data.DepTime = data.DepTime.str.split(":")
data["DepHour"] = data.DepTime.str[0]
data["DepMinute"] = data.DepTime.str[1]

data.ArrTime = data.ArrTime.str.split(":")
data["ArrHour"] = data.ArrTime.str[0]
data["ArrMinute"] = data.ArrTime.str[1]

data = pd.concat((data, pd.get_dummies(data["Day"])), axis=1)
#data = pd.concat((data, pd.get_dummies(data["Way"])), axis=1)
data = pd.concat((data, pd.get_dummies(data["Airplane"])), axis=1)
data = pd.concat((data, pd.get_dummies(data["DepCode"])), axis=1)
data = pd.concat((data, pd.get_dummies(data["DesCode"])), axis=1)

In [3]:
data["BoughtDay"] = data["BoughtDay"].astype('int64')
data["BoughtMonth"] = data["BoughtMonth"].astype('int64')
data["BoughtYear"] = data["BoughtYear"].astype('int64')
data["FlightDay"] = data["FlightDay"].astype('int64')
data["FlightMonth"] = data["FlightMonth"].astype('int64')
data["FlightYear"] = data["FlightYear"].astype('int64')
data["DepHour"] = data["DepHour"].astype('int64')
data["DepMinute"] = data["DepMinute"].astype('int64')
data["ArrHour"] = data["ArrHour"].astype('int64')
data["ArrMinute"] = data["ArrMinute"].astype('int64')
data["Stop"] = data["Stop"].astype('float64')

In [4]:
dependent_variable = "Price"
independent_variables = data.columns.tolist()
independent_variables.remove(dependent_variable)
independent_variables.remove("Date")
independent_variables.remove("DateOfBought")
independent_variables.remove("DepCode")
independent_variables.remove("DesCode")
independent_variables.remove("PriceEx")
independent_variables.remove("Way")
independent_variables.remove("Day")
independent_variables.remove("Stop")
independent_variables.remove("Airplane")
independent_variables.remove("ArrTime")
independent_variables.remove("DepTime")

data.shape

(962, 66)

In [5]:
x = data[independent_variables].values
y = data[dependent_variable].values

x = np.nan_to_num(x)
y = np.nan_to_num(y)

x = np.array(x)
y = np.array(y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [7]:
# scaling datas
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Linear Regresion
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

l_mean_squared_error = math.sqrt(mean_squared_error(y_test, y_pred))
l_r2_score = r2_score(y_test, y_pred)
l_adjusted_rand_score = adjusted_rand_score(y_test, y_pred)

# Print scores
print("Lin Reg RMSE: ", str(l_mean_squared_error))
print("Lin Reg R2_SCORE: ", str(l_r2_score))

# Random Forest Regression
rfr = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)

r_mean_squared_error = math.sqrt(mean_squared_error(y_test, y_pred))
r_r2_score = r2_score(y_test, y_pred)
r_adjusted_rand_score = adjusted_rand_score(y_test, y_pred)

print("Ran Reg RMSE: ", str(r_mean_squared_error))
print("Ran Reg R2_SCORE: ", str(r_r2_score))

# Multiple perceptron regression
mpr = MLPRegressor(hidden_layer_sizes=(100, 100,), activation = "identity", max_iter=1000)
mpr.fit(x_train, y_train)
y_pred = mpr.predict(x_test)

m_mean_squared_error = math.sqrt(mean_squared_error(y_test, y_pred))
m_r2_score = r2_score(y_test, y_pred)
m_adjusted_rand_score = adjusted_rand_score(y_test, y_pred)

print("MP Reg RMSE: ", str(m_mean_squared_error))
print("MP Reg R2_SCORE: ", str(m_r2_score))

# KNN Regression
knr = KNeighborsRegressor(n_neighbors=3)
knr.fit(x_train, y_train)
y_pred = knr.predict(x_test)

k_mean_squared_error = math.sqrt(mean_squared_error(y_test, y_pred))
k_r2_score = r2_score(y_test, y_pred)
k_adjusted_rand_score = adjusted_rand_score(y_test, y_pred)

print("KNN Reg RMSE: ", str(k_mean_squared_error))
print("KNN Reg R2_SCORE: ", str(k_r2_score))


# SVM Regression
svr = SVR(kernel="linear")
svr.fit(x_train, y_train)
y_pred = svr.predict(x_test)

s_mean_squared_error = math.sqrt(mean_squared_error(y_test, y_pred))
s_r2_score = r2_score(y_test, y_pred)
s_adjusted_rand_score = adjusted_rand_score(y_test, y_pred)

print("SV Reg RMSE: ", str(s_mean_squared_error))
print("SV Reg R2_SCORE: ", str(s_r2_score))

Lin Reg RMSE:  136.38315446416792
Lin Reg R2_SCORE:  0.7060238393229264
Ran Reg RMSE:  81.69224487390991
Ran Reg R2_SCORE:  0.8945243252649695
MP Reg RMSE:  137.3756075899357
MP Reg R2_SCORE:  0.701729773636981
KNN Reg RMSE:  144.66927522460554
KNN Reg R2_SCORE:  0.6692169382636196
SV Reg RMSE:  219.3327902286444
SV Reg R2_SCORE:  0.2396776125632868
